In [1]:
%reload_ext autoreload
%aimport -pandas, -numpy
%autoreload 2
import pandas as pd; pd.options.mode.chained_assignment = None # default warn => SettingWithCopyWarning
import numpy as np
from datetime import datetime
from pprint import pprint as pp
import warnings; warnings.simplefilter(action='ignore', category=UserWarning)
from IPython.display import HTML, display
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, IntSlider
import ipywidgets as widgets
import arrow
from tqdm.auto import tqdm

#import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine, 
    msgFSM, filterFSM, FSM_splot, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings, FSMPlot_Start, 
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, 
    disp_result, alarms_pareto, warnings_pareto, states_lines,
    detect_edge_right, detect_edge_left, cvset, load_data, get_cycle_data, get_cycle_data2, figures)
    
cred()
mp = MyPlant(0)
Engine._list_cached_validations();
#mp._fetch_installed_base(); # refresh local installed fleet database

In [2]:
def sfun(x):
    return all([ ("EWB" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Pforzheim" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("BMW Landshut 4.10" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T

,0,1,2,3
Count_OpHour,25000.0,84280.0,35151.0,35352.0
OperationalCondition,Never Connected,Never Connected,Running,Running
shutdown_counter,NaN,NaN,803.0,1132.0
startup_counter,NaN,NaN,801.0,1133.0
id,25475,108005,108141,108528
Engine Version,B227,E01,F12,F12
Engine Type,620,620,620,620
Engine Series,6,6,6,6
Country,IE,US,DE,DE
Commissioning Date,2011-05-17,2003-09-18,2017-04-11,2017-04-11


In [4]:
motor = fleet.iloc[2]
modes = ['???','OFF','MANUAL','AUTO']
success = [True,False]
#success = [True]
e=Engine.from_fleet(mp,motor)
#pp_from=e['Commissioning Date']
pp_from='2021-01-01'
pp_to='2022-03-30'
#pp_to=datetime.now()
fsm = msgFSM(e, p_from=pp_from, p_to=pp_to)
#TODO:
# - move successtime to run1
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

('EWB BAUTZEN M01', ['???', 'OFF', 'MANUAL', 'AUTO'], [True, False])

### den queue algorithmus entwerfen

In [5]:
def consume_testmessage_queue(message_queue):
    return []

In [6]:
def fill_testmessage_queue(mq, d, dt):
    z = next(d)
    zr = z
    while zr < z+dt:
        mq.append(zr)
        try:
            zr = next(d)
        except StopIteration:
            mq.pop()
            break
    mq.append(zr)
    return mq

In [7]:
def merge_extra_testmessages(mq, em):
    emc = em.copy()
    for e in emc:
        if e < max(mq):
            mq.append(e)
            em.remove(e)
    mq = sorted(mq)
    return mq, em

In [8]:
d = list(range(0,56,2)).__iter__()
mq = []
em = [3,7, 37,53,21]

while True:
    try:
        mq = fill_testmessage_queue(mq, d, 20)
        print('after fillmessage:    ',mq)
        print('extramessages:        ',em)
        mq, em = merge_extra_testmessages(mq, em)
        print('merge extramessages:  ',mq)
        print('extramessages:        ',em)
        mq = consume_testmessage_queue(mq)
        print('after consume mgs:    ',mq)
        print()
    except StopIteration:
        print(f"Ende.")
        break

after fillmessage:     [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
extramessages:         [3, 7, 37, 53, 21]
merge extramessages:   [0, 2, 3, 4, 6, 7, 8, 10, 12, 14, 16, 18, 20]
extramessages:         [37, 53, 21]
after consume mgs:     []

after fillmessage:     [22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42]
extramessages:         [37, 53, 21]
merge extramessages:   [21, 22, 24, 26, 28, 30, 32, 34, 36, 37, 38, 40, 42]
extramessages:         [53]
after consume mgs:     []

after fillmessage:     [44, 46, 48, 50, 52, 54]
extramessages:         [53]
merge extramessages:   [44, 46, 48, 50, 52, 53, 54]
extramessages:         []
after consume mgs:     []

Ende.


### mit echten messages:

In [115]:
import pandas as pd

In [116]:
def fill_message_queue(mq, d, dt):
    """fills a message queue with dt time lenght

    Args:
        mq (list): message que
        d (iterator): message iterator variable
        dt (int): delta t in min

    Returns:
        _type_: _description_
    """
    i, m = next(d)
    t0 = m['timestamp'] 
    #print('von: ',t0, pd.to_datetime(t0 * 1000000))
    dtns = dt*60*1000
    #print('bis: ',t0 + dtns, pd.to_datetime((t0 + dtns) * 1000000))
    mit = m
    while mit['timestamp'] < t0 + dtns:
        mq.append(mit)
        try:
            i, mit = next(d)
        except StopIteration:
            mq.pop()
            break
    mq.append(mit)
    return mq

In [117]:
def merge_extra_messages(mq, em):
    emc = em.copy()
    max_timestamp = max([ts['timestamp'] for ts in mq])
    for m in emc:
        if m['timestamp'] < max_timestamp:
            mq.append(m)
            em.remove(m)
    mq.sort(key=lambda x: x['timestamp'], reverse=False)
    return mq, em

In [118]:
def consume_message_queue(message_queue):
    # run the finite state machine
    return []

In [123]:
from extramessages import extra_messages
em = extra_messages.copy()
it = fsm._messages.iterrows()
mq = []
#.strftime('%Y-%m-%d %H:%M:%S')

In [124]:
def debug_msg(msgque, max=10):
    return
    print(len(msgque))
    for msg in msgque[:max]:
        print(f"{msg['timestamp']} {pd.to_datetime(msg['timestamp'] * 1000000)} {msg['name']} {msg['message']}")
    print()

In [125]:
from tqdm.auto import tqdm

pbar = tqdm(total=len(fsm._messages), ncols=80, mininterval=1, unit=' messages', desc="FSM")
#tqdm(self._messages.iterrows(), total=self._messages.shape[0], ncols=80, mininterval=1, unit=' messages', desc="FSM"):

count = 0
while True:
    try:
        mq = fill_message_queue(mq, it, 5*60)
        debug_msg(mq)
        debug_msg(em)
        count += len(mq)
        pbar.update(count)

        mq, em = merge_extra_messages(mq, em)
        debug_msg(mq)
        debug_msg(em)

        mq = consume_message_queue(mq)
        debug_msg(mq)
    except StopIteration:
        print(f"{len(fsm._messages)} messages scanned.")
        break

pbar.close()

FSM: 30428426 messages [00:01, 19177658.22 messages/s]                          

32195 messages scanned.


### first playground for message queue

In [ ]:

m = next(it)
t0 = m['timestamp']
message_queue.append(m)

while m['timestamp'] < t0 + 1*60*60*1000: # 1h
    t = next(it)
    message_queue.append(t)

message_queue = consume_message_queue(message_queue)

In [ ]:
t0 = message_queue[0][1]['timestamp']
for m in message_queue:
    print(f"{m[0]:04} {m[1]['timestamp']:10d} {(m[1]['timestamp'] - t0):10d} sec {m[1]['name']} {m[1]['message']}")

In [ ]:
itt = fsm._messages.itertuples()

In [ ]:
t = next(itt)
t.timestamp, t.name, t.message